In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.fft import fft, fftfreq 
from scipy import fft
import scipy.io
from scipy import signal

from adtk.data import validate_series
from adtk.visualization import plot
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from get_Data import getData,getSample
from myplot import myplot1


In [ ]:
XData1, YData1 = getData("bearing/K005/","N15_M07_F10_K005_")
XData2, YData2 = getData("bearing/KA05/","N15_M07_F10_KA05_")
XData3, YData3 = getData("bearing/KI05/","N15_M07_F10_KI05_")    
XData4, YData4 = getData("bearing/KB27/","N15_M07_F10_KB27_") 
lx = np.linspace(0,80, 5120000)
XData1 = XData1[0:len(lx)]
YData1 = YData1[0:len(lx)]
YData2 = YData2[0:len(lx)]
YData3 = YData3[0:len(lx)]
YData4 = YData4[0:len(lx)]
tt = XData1

In [ ]:
X1,Y1 = getSample("bearing/K004/","N15_M07_F10_K004_1")
X2,Y2 = getSample("bearing/KA04/","N15_M07_F10_KA04_1")
X3,Y3 = getSample("bearing/KI18/","N15_M07_F10_KI18_1")
X4,Y4 = getSample("bearing/KB27/","N15_M07_F10_KB27_1")
lx = np.linspace(0,4,256000)
Y1 = Y1[0:len(lx)]
Y2 = Y2[0:len(lx)]
Y3 = Y3[0:len(lx)]
Y4 = Y4[0:len(lx)]


In [ ]:
def removeOutlier1(tt,data,anomalies,window):
    #print(len(tt),len(data))
    ano1 = anomalies.iloc[0:,0]
    dataA = data.iloc[0:,0]
    dataMean = np.zeros(len(tt))
    dataMedian = np.zeros(len(tt))
    dataNan = np.zeros(len(tt))

    for i in range(0,len(tt)):
        dataMean[i]=dataA[i]
        dataMedian[i]=dataA[i]
        dataNan[i] = dataA[i]
        if (ano1[i]==True and i >=window and i <len(tt)-window):
            mean = np.mean(dataA[i-window:i+window])
            median = np.median(dataA[i-window:i+window])
            dataMean[i]=mean
            dataMedian[i]=median
            dataNan[i]= np.nan
    return dataMean,dataMedian,dataNan

def removeOutlier(tt,data,anomalies,window):
    #print(len(tt),len(data))
    ano1 = anomalies.iloc[0:,0]
    dataA = data.iloc[0:,0]
    dataMean = np.zeros(len(tt))
    dataMedian = np.zeros(len(tt))
    dataNan = np.zeros(len(tt))

    mean = np.mean(dataA[0:])
    median = np.median(dataA[0:])
    for i in range(0,len(tt)):
        dataMean[i]=dataA[i]
        dataMedian[i]=dataA[i]
        dataNan[i] = dataA[i]
        
        if (ano1[i]==True):
            dataMean[i]=mean
            dataMedian[i]=median
            dataNan[i]= np.nan
    return dataMean,dataMedian,dataNan

def removeOutlier2(tt,data,anomalies):
    #print(len(tt),len(data))
    ano1 = anomalies.iloc[0:,0]
    ano2 = anomalies.iloc[0:,1]
    dataA = data.iloc[0:,0]
    dataB = data.iloc[0:,1]
    dataAN = dataA
    dataBN = dataB
    aa = []
    at = []
    bb = []
    bt = []
    for i in range(0,len(tt)):
        if (ano1[i]==True and i >=150 and i <len(tt)-150):
            mean = np.mean(dataA[i-150:i+150])
            dataAN[i]=mean
        if (ano2[i]==True and i >=150 and i <len(tt)-150):
            mean = np.mean(dataB[i-150:i+150])
            dataBN[i]=mean
    return [dataAN,dataBN]
    

In [ ]:

time = pd.to_datetime(lx,unit="s")
#ttime = pd.to_timedelta(tt,unit="milliseconds")
#ttime.to_pytimedelta()
df = pd.DataFrame(index=time, data = {"healthy": Y1,"broken":Y2})
s = validate_series(df)
df.head()


In [ ]:


def myplot2(tt,data, anomalies):
    
    ano1 = anomalies.iloc[0:,0]  
    dataA = data.iloc[0:,0]
    aa = []
    at = []
    anom = 0  
    
    for i in range(0,len(tt)):
        
        if ano1[i]==True:
            aa.append(dataA[i])
            at.append(tt[i])
            anom = anom +1
    
    sns.set(rc= {"figure.figsize": (10,5)})
    fig, ax1 = plt.subplots(nrows=1,ncols=1,figure= (10,5))
    ax1.plot(tt,data["healthy"],marker=".",color = "blue",label="Gesund",zorder=1)
    ax1.scatter(at,aa,marker = "o",color="red",label="Ausreißer",zorder=2)
    ax1.set_ylabel("Beschleunigung in [m/$s^2$]")
    ax1.set_xlabel("Zeit in [s]")
    ax1.legend()
    plt.show()
    return anom
